In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)#데이터 내려 받고 레이블을 원핫인코딩방식으로 읽는다.

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
X = tf.placeholder(tf.float32, [None,784]) # None은 크기가 정해지지 않았다는 것, 한번에 학습시킬 MNIST 이미지 개수 지정 값
Y = tf.placeholder(tf.float32, [None,10]) # 배치 크기 미리 정해도 되지만 학습 개수 계속 바꿔가면서 실험하려는 경우 None넣으면 알아서 계산

In [5]:
# 신경망 구축
W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X,W1))

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)

In [6]:
# 손실값 구하고 평균 송실값 구하기, 최적화 수행하도록 그래프 구성
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [7]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [8]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [9]:
for epoch in range(15):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict={X:batch_xs, Y:batch_ys})
        total_cost += cost_val
    print('Epoch:','%04d' % (epoch+1),
         'Avg.cost = ', '{:3f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg.cost =  0.409798
Epoch: 0002 Avg.cost =  0.154584
Epoch: 0003 Avg.cost =  0.098350
Epoch: 0004 Avg.cost =  0.069122
Epoch: 0005 Avg.cost =  0.052749
Epoch: 0006 Avg.cost =  0.039852
Epoch: 0007 Avg.cost =  0.030313
Epoch: 0008 Avg.cost =  0.024936
Epoch: 0009 Avg.cost =  0.020489
Epoch: 0010 Avg.cost =  0.019497
Epoch: 0011 Avg.cost =  0.015210
Epoch: 0012 Avg.cost =  0.011715
Epoch: 0013 Avg.cost =  0.014546
Epoch: 0014 Avg.cost =  0.010252
Epoch: 0015 Avg.cost =  0.012056
최적화 완료!


In [10]:
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))

In [11]:
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [12]:
print('정확도:',sess.run(accuracy,
                     feed_dict={X:mnist.test.images,
                               Y:mnist.test.labels}))

정확도: 0.9749
